# Versión mejorada

In [5]:
import numpy as np
import random
from joblib import Parallel, delayed

# Lectura de datos desde archivo
data = np.loadtxt('qap.datos/tai256c.dat', skiprows=1)

# Separación en matrices de flujos y distancias
flow = np.int32(data[:256])
distances = np.int32(data[256:])

# Configuración de la semilla para reproducibilidad
seed = 2024
random.seed(seed)
np.random.seed(seed)

# Generación de población inicial de 2000 individuos
population = np.array([np.random.permutation(np.arange(256)) for _ in range(2000)])

# Función de fitness eficiente usando operaciones matriciales
def fitness_pop(population, flow, distances):
    return np.sum(flow[np.newaxis, :, :] * distances[population[:,:,np.newaxis], population[:, np.newaxis, :]], axis=(1,2))

# Selección por torneo con elitismo
def selection_tournament_with_elitism(population, fitness_vals, k=3, elite_ratio=0.1):
    elite_size = int(len(population) * elite_ratio)
    elite_indices = np.argsort(fitness_vals)[:elite_size]
    elites = population[elite_indices]

    selected_indices = np.array([random.choices(np.arange(len(fitness_vals)), k=k, weights=1/fitness_vals)[0] for _ in range(len(population) - elite_size)])
    selected = population[selected_indices]

    return np.vstack((elites, selected))

# Cruce basado en ciclos (Cycle Crossover, CX)
def crossover_cycle(parent1, parent2):
    n = len(parent1)
    child = -np.ones(n, dtype=int)
    start = 0
    indices = set()

    while start not in indices:
        indices.add(start)
        start = np.where(parent2 == parent1[start])[0][0]

    for idx in indices:
        child[idx] = parent1[idx]

    for idx in range(n):
        if child[idx] == -1:
            child[idx] = parent2[idx]

    return child

def crossover_population(population, crossover_prob=0.9):
    new_population = []
    for _ in range(len(population) // 2):
        parent1, parent2 = population[random.sample(range(len(population)), 2)]
        if random.random() < crossover_prob:
            child1 = crossover_cycle(parent1, parent2)
            child2 = crossover_cycle(parent2, parent1)
        else:
            child1, child2 = parent1.copy(), parent2.copy()
        new_population.extend([child1, child2])
    return np.array(new_population)

# Mutación adaptativa por intercambio
def mutate_population_adaptive(population, mutation_prob, stagnation_counter, max_stagnation):
    adaptive_prob = mutation_prob + (0.1 * stagnation_counter / max_stagnation)
    adaptive_prob = min(adaptive_prob, 0.5)  # Limitar la mutación adaptativa a un máximo de 50%

    for individual in population:
        if random.random() < adaptive_prob:
            i, j = random.sample(range(len(individual)), 2)
            individual[i], individual[j] = individual[j], individual[i]
    return population

# Búsqueda local (2-opt limitada y paralelizada)
def local_search_2opt_limited(permutation, flow, distances, max_iter=20):
    n = len(permutation)
    best_cost = calculate_cost(permutation, flow, distances)
    for _ in range(max_iter):
        i, j = sorted(random.sample(range(n), 2))
        new_perm = permutation.copy()
        new_perm[i:j+1] = list(reversed(new_perm[i:j+1]))
        new_cost = calculate_cost(new_perm, flow, distances)
        if new_cost < best_cost:
            permutation = new_perm
            best_cost = new_cost
    return permutation

def parallel_local_search(population, flow, distances, max_iter=20, elite_ratio=0.1):
    elite_size = int(len(population) * elite_ratio)
    elite_indices = np.argsort(fitness_pop(population, flow, distances))[:elite_size]
    elite_population = population[elite_indices]

    improved_elites = Parallel(n_jobs=-1)(
        delayed(local_search_2opt_limited)(individual, flow, distances, max_iter)
        for individual in elite_population
    )

    population[elite_indices] = improved_elites
    return population

# Calcular el costo
def calculate_cost(permutation, flow, distances):
    indexed_distances = distances[permutation][:, permutation]
    cost = np.sum(flow * indexed_distances)
    return cost

# Ciclo evolutivo
generations = 200
crossover_prob = 0.9
mutation_prob = 0.3
stagnation_limit = 40  # Número de generaciones sin mejora para detener

best_solution = None
best_fitness = float('inf')
no_improvement_counter = 0

fitness_vals = fitness_pop(population, flow, distances)

for generation in range(generations):
    # Introducir nuevos individuos si hay estancamiento
    if no_improvement_counter >= stagnation_limit // 2:
        new_individuals = np.array([np.random.permutation(np.arange(256)) for _ in range(len(population) // 10)])
        population = np.vstack((population, new_individuals))

    # Búsqueda local paralelizada en la élite
    population = parallel_local_search(population, flow, distances, elite_ratio=0.1)

    # Selección con elitismo
    selected_population = selection_tournament_with_elitism(population, fitness_vals)

    # Cruce
    offspring = crossover_population(selected_population, crossover_prob)

    # Mutación adaptativa
    population = mutate_population_adaptive(offspring, mutation_prob, no_improvement_counter, stagnation_limit)

    # Recalcular fitness
    fitness_vals = fitness_pop(population, flow, distances)

    # Guardar el mejor individuo
    gen_best_fitness = fitness_vals.min()
    if gen_best_fitness < best_fitness:
        best_fitness = gen_best_fitness
        best_solution = population[np.argmin(fitness_vals)]
        no_improvement_counter = 0
    else:
        no_improvement_counter += 1

    print(f"Generación {generation + 1}: Mejor Fitness = {best_fitness}")

    # Criterio de parada por estancamiento
    if no_improvement_counter >= stagnation_limit:
        print("Estancamiento detectado, deteniendo el proceso.")
        break

print("Mejor solución encontrada:", best_solution)
print("Costo asociado:", best_fitness)


Generación 1: Mejor Fitness = 49988314
Generación 2: Mejor Fitness = 49617080
Generación 3: Mejor Fitness = 49617080
Generación 4: Mejor Fitness = 49309794
Generación 5: Mejor Fitness = 49309794
Generación 6: Mejor Fitness = 49309794
Generación 7: Mejor Fitness = 49235692
Generación 8: Mejor Fitness = 49235692
Generación 9: Mejor Fitness = 49235692
Generación 10: Mejor Fitness = 49235692
Generación 11: Mejor Fitness = 49235692
Generación 12: Mejor Fitness = 49235692
Generación 13: Mejor Fitness = 49235692
Generación 14: Mejor Fitness = 49235692
Generación 15: Mejor Fitness = 49235692
Generación 16: Mejor Fitness = 49235692
Generación 17: Mejor Fitness = 49217256
Generación 18: Mejor Fitness = 49217256
Generación 19: Mejor Fitness = 49217256
Generación 20: Mejor Fitness = 49217256
Generación 21: Mejor Fitness = 49217256
Generación 22: Mejor Fitness = 49217256
Generación 23: Mejor Fitness = 49217256
Generación 24: Mejor Fitness = 49217256
Generación 25: Mejor Fitness = 49217256
Generació